In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pip install bitarray
!pip install keras
!pip install scikit-learn
!pip install mmh3
!pip install keras==2.1.3


  Created wheel for bitarray: filename=bitarray-1.0.1-cp36-cp36m-linux_x86_64.whl size=76940 sha256=22bbbb653a22656decdf7b424657ee2c6496ef4b3c345899316f7f7d7fc155c7
  Stored in directory: /root/.cache/pip/wheels/d2/e3/1b/82b4f8f9e4f36c6b3f171b1781d5a7d85e74a113f1ee58f0f4
Successfully built bitarray
  Created wheel for mmh3: filename=mmh3-2.5.1-cp36-cp36m-linux_x86_64.whl size=37866 sha256=e8502774bc6ed1094be9701853ceca8a2fb9419ec647d38dafc5109775e5e9e7
  Stored in directory: /root/.cache/pip/wheels/38/b4/ea/6e4e321c625d3320c0c496bf4088371546d8fce5f1dd71b219
Successfully built mmh3
     |████████████████████████████████| 327kB 2.9MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.1.3 which is incompatible.
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [4]:
import sys
sys.path.append('/content/drive/My Drive/DeepBloom-master/lib')
sys.path.append('/content/drive/My Drive/DeepBloom-master')
#from GRUModel import GRUModel
#from DeepBloom import DeepBloom
#from DeeperBloom import DeeperBloom
import json
from utils import *
#from test_deep_bloom import test_gru_model
#from test_bloom_filter import url_test

with open('/content/drive/My Drive/DeepBloom-master/data/dataset.json', 'r') as f:
	    dataset = json.load(f)

positives = dataset['positives']
negatives = dataset['negatives']

positives = positives[:]
negatives = negatives[:]
print(len(positives), len(negatives))
print("Baseline")

1491178 1435527
Baseline


In [5]:
from keras.models import Model
from utils import *
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Flatten
from keras.layers import LSTM, Input, GRU
from keras import optimizers
from sklearn.decomposition import PCA
import numpy as np
from keras import backend as K
from keras.regularizers import l2
from keras.utils.generic_utils import get_custom_objects
from keras import losses
from keras.engine.topology import Layer
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
from keras.layers.advanced_activations import PReLU
from keras import initializers

def prelu(x, name='default'):
    if name == 'default':
        return PReLU(alpha_initializer=initializers.Constant(value=0.25))(x)
    else:
        return PReLU(alpha_initializer=initializers.Constant(value=0.25), name=name)(x)

      
def i2str(i):
    s = str(i)
    if len(s) == 1:
        return '0' + s
    else:
        return s
      
def focal_loss(alpha_p,alpha_n,gama):

    def loss(y_true,y_pred):
        y_p = 1-y_pred+y_true*(2*y_pred-1)
        alpha = alpha_p*y_true+alpha_n*(1-y_true)
        return -K.mean(alpha*K.pow(1-y_p,gama)*K.log(y_p))
    return loss

def focal_loss_plus1(alpha_p,alpha_n,gama):

    def loss(y_true,y_pred):
        y_p = 1-y_pred+y_true*(2*y_pred-1)
        alpha = alpha_p*y_true+alpha_n*(1-y_true)
        k = K.pow(1-y_p,gama)*y_true+(1-y_true)
        return -K.mean(alpha*k*K.log(y_p))
    return loss

def focal_loss_plus2(alpha_p,alpha_n,gama):

    def loss(y_true,y_pred):
        y_p = 1-y_pred+y_true*(2*y_pred-1)
        alpha = alpha_p*y_true+alpha_n*(1-y_true)
        k = K.pow(1-y_p,gama)*(1-y_true)+y_true
        return -K.mean(alpha*k*K.log(y_p))
    return loss

def focal_loss_plusS(alpha_p,alpha_n,gama,thr_p,thr_n):

    def loss(y_true,y_pred):
        y_p = 1-y_pred+y_true*(2*y_pred-1)
        alpha = alpha_p*y_true+alpha_n*(1-y_true)
        k = K.pow(1-y_p,gama)
        y_all = y_true+1-y_true
        k_p = K.switch(y_p>thr_p,k,y_all)
        k_n = K.switch(y_p>thr_n,k,y_all)
        k = k_p*y_true+k_n*(1-y_true)
        return -K.mean(alpha*k*K.log(y_p))
    return loss

def custom_activation(x):
    kp = 1
    kn = 2
    y = K.switch(x>0,kp*x,kn*x)
        
    #return (K.sigmoid(1/(1+K.exp(K.max(x,0)+K.min(x,0)))))
    return (K.sigmoid(y))

class CenterLossLayer(Layer):

    def __init__(self, alpha=0.5, **kwargs):
        super().__init__(**kwargs)
        self.alpha = alpha

    def build(self, input_shape):
        self.centers = self.add_weight(name='centers',
                                       shape=(2, 2),
                                       initializer='uniform',
                                       trainable=False)
        # self.counter = self.add_weight(name='counter',
        #                                shape=(1,),
        #                                initializer='zeros',
        #                                trainable=False)  # just for debugging
        super().build(input_shape)

    def call(self, x, mask=None):

        # x[0] is Nx2, x[1] is Nx10 onehot, self.centers is 10x2
        delta_centers = K.dot(K.transpose(x[1]), (K.dot(x[1], self.centers) - x[0]))  # 10x2
        center_counts = K.sum(K.transpose(x[1]), axis=1, keepdims=True) + 1  # 10x1
        delta_centers /= center_counts
        new_centers = self.centers - self.alpha * delta_centers
        self.add_update((self.centers, new_centers), x)
        weight = np.array([-1,1]).reshape(-1,2)
        weight_k = K.variable(value=weight,dtype='float32')
        center_dis = K.dot(weight_k,self.centers)
        center_dis = K.sum(center_dis ** 2)
        # self.add_update((self.counter, self.counter + 1), x)

        self.result = x[0] - K.dot(x[1], self.centers)
        self.result = K.sum(self.result ** 2, axis=1, keepdims=True) / K.dot(x[1], center_counts)
        self.result = self.result/center_dis
        return self.result # Nx1

    def compute_output_shape(self, input_shape):
        return K.int_shape(self.result)

def zero_loss(y_true, y_pred):
    return 0.5 * K.sum(y_pred, axis=0)

def my_visualize_focal(X, y,a,epoch, threshold,ap,an,gama,time,fnr,fpr,lossfn,thr_n,thr_p):
    c = ['#ff0000', '#ffff00']
    plt.figure()
    plt.clf()
    X = np.asarray(X).reshape(-1,2)
    b = np.where(y==0)[0]
    c = np.where(y==1)[0]
    # print(b[:10])
    d = set(a)&set(c)
    d = list(d)
    e = set(a)&set(b)
    e = list(e)
    plt.plot(X[b,0], X[b,1], '.', color = 'red')
    plt.plot(X[c,0], X[c,1], '.', color = 'yellow')
    plt.plot(X[d,0], X[d,1], '.', color = 'blue')
    plt.plot(X[e,0],X[e,1], '.', color = 'green')
    legend_elements = [Line2D([0], [0], color="white",markerfacecolor='yellow',marker='o', label='1'),
              Line2D([0], [0], color="white",markerfacecolor='red',marker='o', label='0'),
              Line2D([0], [0], color="white",markerfacecolor='green',marker='o', label='f'),
              Line2D([0], [0], color="white",markerfacecolor='blue',marker='o', label='t')]
    plt.title('Thr-{:.3f}'.format(threshold)+'-Ap-{}-An-{}-Gama-{}'.format(ap,an,gama,fnr)+'-FPR-{:.3f}'.format(fpr)+'tp-{}-tn-{}'.format(thr_n,thr_p))
    plt.legend(handles=legend_elements, loc='upper right')
    plt.savefig('/content/drive/My Drive/DeepBloom-master/data/EX2-'+lossfn+'-epoch-{}-Ap-{}-An-{}-Gama-{}-Time-{}-Tp-{}-Tn-{}-train.png'.format(i2str(epoch), ap,an,gama,time,thr_n,thr_p))
    plt.close()

def draw_hist(data,title,filepath):
  plt.figure()
  plt.clf()
  plt.hist(data, bins = 20)
  plt.title(title)
  plt.savefig(filepath)
  plt.close()

def my_visualize(feat, labels, epoch, lambda_cl):
    c = ['#ff0000', '#ffff00']
    plt.figure()
    plt.clf()
    center_x = [0,0]
    center_y = [0,0]
    for i in range(2):
        index = np.where(labels==i)[0]
        for ind in index:
          plt.plot(feat[ind][0], feat[ind][1], '.', c=c[i])
          center_x[i] = feat[ind][0]+center_x[i]
          center_y[i] = feat[ind][1]+center_y[i]
        center_x[i] = center_x[i]/len(index)
        center_y[i] = center_y[i]/len(index)
    legend_elements = [Line2D([0], [0], color="white",markerfacecolor=c[0],marker='o', label='0'),
                Line2D([0], [0], color="white",markerfacecolor=c[1],marker='o', label='1')]
    plt.plot(center_x, center_y, 'kx', mew=2, ms=4)
    plt.title('New loss Training data. Lambda_centerloss = {}, Epoch = {}'.format(lambda_cl, epoch))
    plt.legend(handles=legend_elements, loc='upper right')
    plt.savefig('/content/drive/My Drive/DeepBloom-master/data/newGSD epoch-{}-lambda-{}-train.png'.format(i2str(epoch), lambda_cl))
    plt.close()
    
get_custom_objects().update({'custom_activation': Activation(custom_activation)})
class GRUModel(Model):
    def __init__(self, embeddings_path, embedding_dim,thr_p=0.5,thr_n=0.5,time=0,lossfn='focal',lambda_centerloss=0.5,alpha_p=1,alpha_n=1,gama=0, lr=0.001, maxlen=50, pca_embedding_dim=None, batch_size=1024, gru_size=16, hidden_size=None, second_gru_size=None, decay=0.0001, epochs=30, lstm=False, dense_only=False,loss_fun = 'binary_crossentropy'):
        self.embeddings_path = embeddings_path
        self.embedding_dim = embedding_dim
        self.lr = lr
        self.maxlen = maxlen
        self.pca_embedding_dim = pca_embedding_dim
        self.model = None
        self.batch_size = batch_size
        self.gru_size = gru_size
        self.hidden_size = hidden_size
        self.second_gru_size = second_gru_size
        self.decay = decay
        self.epochs = epochs
        self.lstm = lstm
        self.dense_only = dense_only
        self.loss_fun = loss_fun
        self.alpha_p = alpha_p
        self.alpha_n = alpha_n
        self.gama = gama
        self.lambda_centerloss= lambda_centerloss
        self.lossfn = lossfn
        self.time = time
        self.thr_p = thr_p
        self.thr_n = thr_n

    def fit(self, text_X, text_y):

        X, y, self.char_indices, self.indices_char = vectorize_dataset(text_X, text_y, self.maxlen)
        num_chars = len(self.char_indices)
        print(X.shape)
        embedding_vectors = {}
        with open(self.embeddings_path, 'r') as f:
            for line in f:
                line_split = line.strip().split(" ")
                vec = np.array(line_split[1:], dtype=float)
                char = line_split[0]
                embedding_vectors[char] = vec

        embedding_matrix = np.zeros((num_chars + 1, self.embedding_dim))
        for char, i in self.char_indices.items():
            embedding_vector = embedding_vectors.get(char)
            assert(embedding_vector is not None)
            embedding_matrix[i] = embedding_vector

        print(embedding_matrix.shape)

        if self.pca_embedding_dim:
            pca = PCA(n_components=self.pca_embedding_dim)
            pca.fit(embedding_matrix[1:])
            embedding_matrix_pca = np.array(pca.transform(embedding_matrix[1:]))
            embedding_matrix_pca = np.insert(embedding_matrix_pca, 0, 0, axis=0)
            print("PCA matrix created")
        
            
        main_input = Input(shape=(self.maxlen,), name='main_input')
        # side_input = Input(shape=(2,),name='side_input')
        
        x = Embedding(num_chars + 1, self.embedding_dim if not self.pca_embedding_dim else self.pca_embedding_dim, input_length=self.maxlen,
    weights=[embedding_matrix] if not self.pca_embedding_dim else [embedding_matrix_pca])(main_input)
        x = GRU(self.gru_size, return_sequences=False if not self.second_gru_size else True)(x)
        x = Dense(self.hidden_size,activation=prelu, name='hidden_output3',kernel_regularizer=l2(self.decay))(x)
        x = Dense(4,activation=prelu, name='hidden_output2',kernel_regularizer=l2(self.decay))(x)
        x = Dense(2,activation=prelu, name='hidden_output1',kernel_regularizer=l2(self.decay))(x)
        prediction = Dense(1,activation='sigmoid', name='output',kernel_regularizer=l2(self.decay))(x)
        # side_output = CenterLossLayer(alpha=0.5, name='centerlosslayer')([x, side_input])
        #prediction = Dense(1,activation=custom_activation, name='output')(x)
        
        self.model = Model(inputs=[main_input],outputs=[prediction])
        self.model.summary()
        optimizer = optimizers.Adam(lr=self.lr, decay=self.decay)
        #optimizer = optimizers.SGD(lr=self.lr, momentum=0.9)
        self.model.compile(optimizer=optimizer, loss=focal_loss_plusS(self.alpha_p,self.alpha_n,self.gama,self.thr_p,self.thr_n),metrics=['accuracy'])

        y_t = np.zeros((len(y),2),dtype=int)
        for j in range(len(y)):
          if y[j]:
            y_t[j,1] = 1
          else:
            y_t[j,0] = 1
        dummy1 = np.zeros((X.shape[0], 1))
        self.model.fit(X,y, batch_size=self.batch_size, epochs=self.epochs, verbose=2)
        self.reduced_model = Model(inputs=self.model.input, outputs=self.model.get_layer('hidden_output1').output)
        # self.real_model = Model(inputs=self.model.input[0], outputs=self.model.get_layer('output').output)
        self.real_model = self.model
        #self.model.save("/content/drive/My Drive/DeepBloom-master/lib/model_new.h5")
        # self.model = load_model('model.h5')


    def predict(self, text_x):
        x = np.zeros((1, self.maxlen), dtype=np.int)
        offset = max(self.maxlen - len(text_x), 0)
        for t, char in enumerate(text_x):
            if t >= self.maxlen:
                break
            x[0, t + offset] = self.char_indices[char]
        pred = self.real_model.predict(x)
        return pred[0][0]

    # Like predict, but you pass in an array of URLs, and it is all
    # vectorized in one step, making it more efficient
    def predicts(self, text_X):
        X = np.zeros((len(text_X), self.maxlen), dtype=np.int)
        for i in range(len(text_X)):
            offset = max(self.maxlen - len(text_X[i]), 0)
            for t, char in enumerate(text_X[i]):
                if t >= self.maxlen:
                    break
                X[i, t + offset] = self.char_indices[char]
        preds = [pred[0] for pred in self.real_model.predict(X)]
        return preds
      
    def visualize_train(self,text_X,text_y,threshold,fnr,predictions_f):
        text_y = np.asarray(text_y).reshape(-1,1)
        # print(text_y[:10])
        X = np.zeros((len(text_X), self.maxlen), dtype=np.int)
        for i in range(len(text_X)):
            offset = max(self.maxlen - len(text_X[i]), 0)
            for t, char in enumerate(text_X[i]):
                if t >= self.maxlen:
                    break
                X[i, t + offset] = self.char_indices[char]
        preds = [pred for pred in self.reduced_model.predict(X)]
        predictions = [pred[0] for pred in self.real_model.predict(X)]
        predictions = np.asarray(predictions).reshape(-1,1)
        # print(predictions[:10])
        a = np.where(predictions>threshold)[0]
        b = np.where(text_y==1)[0]
        c = list(set(a)&set(b))
        fpr = 1-len(c)/len(b)
        print('fpr:{}'.format(fpr))
        title = 'HNt-'+'ap-{}-an-{}-gama-{}-tp-{}-tn-{}'.format(self.alpha_p,self.alpha_n,self.gama,self.thr_p,self.thr_n)+'-FPR-{:.4f}'.format(fpr)+'-Thr-{:.3f}'.format(threshold)
        filepath = '/content/drive/My Drive/DeepBloom-master/data/'+'EX2-Histogram-Negt-loss-'+self.lossfn+'-time-{}-ap-{}-an-{}-gama-{}-Epoch-{}-FNR-{}-Tp-{}-Tn-{}-train.png'.format(self.time,self.alpha_p,self.alpha_n,self.gama,self.epochs, fnr,self.thr_p,self.thr_n)
        draw_hist(predictions_f,title,filepath)
        title = 'HN-'+'ap-{}-an-{}-gama-{}-tp-{}-tn-{}'.format(self.alpha_p,self.alpha_n,self.gama,self.thr_p,self.thr_n)+'-FPR-{:.4f}'.format(fpr)+'-Thr-{:.3f}'.format(threshold)
        filepath = '/content/drive/My Drive/DeepBloom-master/data/'+'EX2-Histogram-Neg-loss-'+self.lossfn+'-time-{}-ap-{}-an-{}-gama-{}-Epoch-{}-FNR-{}-Tp-{}-Tn-{}-train.png'.format(self.time,self.alpha_p,self.alpha_n,self.gama,self.epochs, fnr,self.thr_p,self.thr_n)
        draw_hist(predictions[np.where(text_y==0)[0]],title,filepath)
        title = 'HP-'+'ap-{}-an-{}-gama-{}-tp-{}-tn-{}'.format(self.alpha_p,self.alpha_n,self.gama,self.thr_p,self.thr_n)+'-FPR-{:.4f}'.format(fpr)+'-Thr-{:.3f}'.format(threshold)
        filepath = '/content/drive/My Drive/DeepBloom-master/data/'+'EX2-Histogram-Pos-loss-'+self.lossfn+'-time-{}-ap-{}-an-{}-gama-{}-Epoch-{}-FNR-{}-Tp-{}-Tn-{}-train.png'.format(self.time,self.alpha_p,self.alpha_n,self.gama,self.epochs, fnr,self.thr_p,self.thr_n)
        draw_hist(predictions[np.where(text_y==1)[0]],title,filepath)
        #my_visualize_focal(preds, text_y,a,self.epochs, threshold,self.alpha_p,self.alpha_n,self.gama,self.time,fnr,fpr,self.lossfn,self.thr_p,self.thr_n)
        # my_visualize_new(preds, text_y, epoch=self.epochs,)
        return preds


Using TensorFlow backend.


In [0]:
from BloomFilter import BloomFilter
import math
import random
from utils import *
import mmh3

class DeepBloom(object):
    def __init__(self, model, data, fp_rate):
        self.model = model
        self.threshold = None
        self.fp_rate = float(fp_rate)
        self.fit(data)
        # self.create_bloom_filter(data)

    def check(self, item):
        if self.model.predict(item) > self.threshold:
            return True
        return self.bloom_filter.check(item)

    def create_bloom_filter(self, data):
        print("Creating bloom filter")
        false_negatives = []
        preds = self.model.predicts(data.positives)
        for i in range(len(data.positives)):
            if preds[i] <= self.threshold:
                false_negatives.append(data.positives[i])
        print("Number of false negatives at bloom time", len(false_negatives))
        self.bloom_filter = BloomFilter(
            len(false_negatives),
            self.fp_rate / 2,
            string_digest
        )
        for fn in false_negatives:
            self.bloom_filter.add(fn)
        print("Created bloom filter")


    def fit(self, data):
        ## Split negative data into subgroups.
        (s1, s2) = split_negatives(data)
        print("Training model with train, dev, positives", len(s1), len(s2), len(data.positives))

        ## Shuffle together subset of negatives and positives.
        ## Then, train the model on this data.
        shuffled = shuffle_for_training(s1, data.positives)
        self.model.fit(shuffled[0], shuffled[1])
        print("Done fitting")

        ## We want a threshold such that at most s2.size * fp_rate/2 elements
        ## are greater than threshold.
        fp_index = math.ceil((len(s2) * (1 - self.fp_rate/2)))
        predictions = self.model.predicts(s2)
        predictions.sort()
        self.threshold = predictions[fp_index]
        self.model.visualize_train(shuffled[0],shuffled[1],self.threshold,self.fp_rate/2,predictions)
        print("done visualization")


In [0]:
def test_gru_model(positives, negatives, model, train_dev_fraction=0.9, deeper_bloom=False, fp_rate=0.01, fp_fractions=None):
    train_dev_negatives = negatives[:int(train_dev_fraction * len(negatives))]
    test_negatives = negatives[int(train_dev_fraction * len(negatives)):]
    print("Number train, dev", len(train_dev_negatives))
    print("Number test", len(test_negatives))
    print("Number positives ", len(positives))

    data = Data(positives, train_dev_negatives)
    if not deeper_bloom:
        db = DeepBloom(model, data, fp_rate)
        print("Params needed", db.model.model.count_params())
    

In [8]:
time = 1
lossfn = 'focal+S' 
thr_p = 0.9
thr_n = 0.5
alpha_p = 1
alpha_n = 7
gama= 2
model = GRUModel('/content/drive/My Drive/DeepBloom-master/data/glove.6B.50d-char.txt', 50,thr_p =thr_p ,thr_n =thr_n ,time=time,lossfn=lossfn,alpha_p = alpha_p,alpha_n =alpha_n,gama=gama, pca_embedding_dim=16, maxlen=40, gru_size=16, batch_size=8192, lr=0.01, hidden_size=8, epochs=40)
test_gru_model(positives, negatives, model, train_dev_fraction=0.95, deeper_bloom=False, fp_rate=0.01)

Number train, dev 1363750
Number test 71777
Number positives  1491178
Training model with train, dev, positives 1227375 136375 1491178
Vectorizing data...
Corpus length 51413132
['-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Total chars: 38
[(4, 26), (5, 478), (6, 4385), (7, 8525), (8, 17841), (9, 29160), (10, 49616), (11, 81456), (12, 130824), (13, 167520), (14, 186210), (15, 189876), (16, 191354), (17, 195244), (18, 193089), (19, 182793), (20, 167004), (21, 146920), (22, 127819), (23, 109895), (24, 92890), (25, 78725), (26, 65687), (27, 54689), (28, 46607), (29, 39943), (30, 33373), (31, 28094), (32, 23714), (33, 19983), (34, 11506), (35, 9527), (36, 7046), (37, 5586), (38, 4419), (39, 3443), (40, 2448), (41, 2035), (42, 1530), (43, 1132), (44, 873), (45, 819), (46, 566), (47, 476), (48, 398), (49, 350), (50, 286), (51, 285), (52, 207), (53, 

In [9]:
time = 1
lossfn = 'focal+S' 
thr_p = 0.9
thr_n = 0.5
alpha_p = 1
alpha_n = 8
gama= 2
model = GRUModel('/content/drive/My Drive/DeepBloom-master/data/glove.6B.50d-char.txt', 50,thr_p =thr_p ,thr_n =thr_n ,time=time,lossfn=lossfn,alpha_p = alpha_p,alpha_n =alpha_n,gama=gama, pca_embedding_dim=16, maxlen=40, gru_size=16, batch_size=8192, lr=0.01, hidden_size=8, epochs=40)
test_gru_model(positives, negatives, model, train_dev_fraction=0.95, deeper_bloom=False, fp_rate=0.01)

Number train, dev 1363750
Number test 71777
Number positives  1491178
Training model with train, dev, positives 1227375 136375 1491178
Vectorizing data...
Corpus length 51413132
['-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Total chars: 38
[(4, 26), (5, 478), (6, 4385), (7, 8525), (8, 17841), (9, 29160), (10, 49616), (11, 81456), (12, 130824), (13, 167520), (14, 186210), (15, 189876), (16, 191354), (17, 195244), (18, 193089), (19, 182793), (20, 167004), (21, 146920), (22, 127819), (23, 109895), (24, 92890), (25, 78725), (26, 65687), (27, 54689), (28, 46607), (29, 39943), (30, 33373), (31, 28094), (32, 23714), (33, 19983), (34, 11506), (35, 9527), (36, 7046), (37, 5586), (38, 4419), (39, 3443), (40, 2448), (41, 2035), (42, 1530), (43, 1132), (44, 873), (45, 819), (46, 566), (47, 476), (48, 398), (49, 350), (50, 286), (51, 285), (52, 207), (53, 